# Sample Sentiment Analysis

In [1]:
import logging
# create logger
logger = logging.getLogger('pypsenti')
logger.setLevel(logging.DEBUG)

# create console handler and set level to debug
ch = logging.StreamHandler()
ch.setLevel(logging.DEBUG)

# create formatter
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')

# add formatter to ch
ch.setFormatter(formatter)
logger.addHandler(ch)

In [2]:
from pypsenti.service.sentiment import SentimentAnalysis, SentimentConnection, Document
from sklearn import metrics
import socket
user_name = socket.gethostname()
connection = SentimentConnection(host='192.168.0.70', port=7044, client_id=user_name)

###  Supported Custom Domains

In [3]:
for domain in connection.supported_domains:
    print(domain)

TwitterMarket
TwitterTrump
market


# Standard Lexicon Sentiment Analysis

In [4]:
%%time
amazon_reviews = ['I love this hello kitty decal! I like that the bow is pink instead of red. Only bad thing is that after putting it on the window theres a few air bubbles, but that most likely my fault. Shipped fast too.',
                  'I bought this for my 3 yr old daughter when I took it out the pack it had a bad oder, cute but very cheap material easy to ripe.  When I tried it on her it was to big, but of course she liked it so I kept it. I dressed her up in it and she looked cute.']

analysis = SentimentAnalysis(connection)

for result in analysis.detect_sentiment_text(amazon_reviews):
    print(f'{result["Stars"]:1.2f}')

2019-11-28 21:53:04,465 - pypsenti.service - INFO - Setting up connection to 192.168.0.70:7044 for hp-z8
2019-11-28 21:53:04,523 - pypsenti.service - DEBUG - Starting loop...
2019-11-28 21:53:04,525 - pypsenti.service - INFO - Sending Analysis Command...
2019-11-28 21:53:04,526 - pypsenti.service - DEBUG - Processing...
2019-11-28 21:53:04,530 - pypsenti.service - INFO - Connected result code {'session present': 0}
2019-11-28 21:53:24,795 - pypsenti.service - INFO - Disconnected result code 1
2019-11-28 21:53:35,765 - pypsenti.service - ERROR - Timeout!


TimeoutError: 

In [4]:
analysis = SentimentAnalysis(SentimentConnection(user_name), model='Test')

for result in analysis.detect_sentiment(documents):
    print(result['Stars'])

NameError: name 'documents' is not defined

In [5]:
%%time
id = 0 
all_amazon_documents = []
true_document_class = {}
with open('../data/amazon/positive.txt', "r", encoding='utf8') as reader:
    for line in reader:
        doc = Document(id, line)
        doc.isPositive = True
        all_amazon_documents.append(doc)
        true_document_class[doc.id] = True
        id += 1
    
with open('../data/amazon/negative.txt', "r", encoding='utf8') as reader:
    for line in reader:
        doc = Document(id, line)    
        doc.isPositive = False
        all_amazon_documents.append(doc)
        true_document_class[doc.id] = False
        id += 1
    
detected_document_class = {}
print(f'Processig {len(all_amazon_documents)} documents...')
analysis = SentimentAnalysis(connection)
for result in analysis.detect_sentiment(all_amazon_documents):
    stars = result['Stars']    
    id  = result['Id']
    detected_document_class[id] = stars is not None and stars > 3
    
print(f'Total processed documents: {len(detected_document_class)}')

test_y = [true_document_class[document.id] for document in all_amazon_documents]
result_y = [detected_document_class[document.id] for document in all_amazon_documents]
vacc = metrics.accuracy_score(test_y, result_y)
print(f'Accuracy: {vacc}')

#

Processig 2000 documents...
Total processed documents: 2000
Accuracy: 0.776
Wall time: 11.4 s


In [6]:
%%time
connection.save_documents('Amazon', all_amazon_documents)
analysis = SentimentAnalysis(connection)

Wall time: 3.14 s


In [7]:
%%time
analysis.train('Amazon')

Wall time: 8.99 s


In [8]:
%%time
analysis = SentimentAnalysis(connection, model='Amazon')
for result in analysis.detect_sentiment(all_amazon_documents):
    stars = result['Stars']    
    id  = result['Id']
    detected_document_class[id] = stars is not None and stars > 3
    
print(f'Total processed documents: {len(detected_document_class)}')

test_y = [true_document_class[document.id] for document in all_amazon_documents]
result_y = [detected_document_class[document.id] for document in all_amazon_documents]
vacc = metrics.accuracy_score(test_y, result_y)
print(f'Accuracy: {vacc}')

Total processed documents: 2000
Accuracy: 0.8895
Wall time: 8.01 s
